In [3]:
import pandas as pd
import geopandas as gpd
from simple_modflow.modflow.mf6.voronoiplus import VoronoiGridPlus as Vor
from pathlib import Path
import shapely as shp
import pickle

class SFR:

    def __init__(
            self,
            vor: Vor = None,
            stream_path: Path = None
    ):
        self.vor = vor
        self.stream_path = stream_path
        self.stream_gdf = gpd.read_file(stream_path)
        self.stream_geom = self.stream_gdf.geometry
        self.stream_points = {}
        self.vor_intersect = {}

        for i, geom in enumerate(self.stream_geom):
            x, y = geom.coords.xy
            self.stream_points[i] = [shp.Point(point) for point in list(zip(x,y))]
            self.vor_intersect[i] = vor.get_vor_cells_as_series(geom)

In [4]:
vor_path = Path(r'C:\Users\lukem\Python\MODFLOW\LakePointe\vor_lakepointe.vor')
with open(vor_path, 'rb') as file:
    vor: Vor = pickle.load(file)

In [5]:
input_path = Path(r'C:\Users\lukem\Python\MODFLOW\LakePointe\inputs')
stream = input_path / r'shp\rivers and streams\streams_simplified.shp'
sfr = SFR(vor=vor, stream_path=stream)

In [54]:
sfr.stream_geom

0    LINESTRING (1244493.689 747168.995, 1244357.98...
1    LINESTRING (1243728.191 742929.328, 1243786.31...
Name: geometry, dtype: geometry

In [18]:
vor_idxs = sfr.vor_intersect[1].to_list()
for idx in vor_idxs:
    cell: gpd.GeoSeries = vor.gdf_vorPolys.loc[idx]
    cell.array.sindex.query(sfr.stream_geom[1])

In [30]:
vor.gdf_vorPolys.loc[idx].array.sindex.query(sfr.stream_geom[1], predicate='intersects')

array([0], dtype=int64)

In [63]:
sfr.stream_geom[1].intersection(vor.gdf_vorPolys.loc[92]).geometry.length

7.999999999975072

In [53]:
vor.gdf_vorPolys.loc[92]

geometry    POLYGON ((1243725.934 742939.512, 1243733.619 ...
Name: 92, dtype: geometry